In [185]:
## setup
import os
import pickle
from keras import backend as K
import cv2
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils.np_utils import to_categorical
import sys
sys.setrecursionlimit(3000)

# path to the model weights file.
weights_path = './keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = './all_ctg_100/train/'
validation_data_dir = './all_ctg_100/validation/'
nb_train_samples = 2400
nb_validation_samples = 240
nb_epoch = 50
nb_category = 24

K.set_image_dim_ordering('th')
datagen = ImageDataGenerator(rescale=1./255)

In [63]:
## create model that transforms image to intermediate stage
model_load = Sequential()
model_load.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model_load.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_load.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_load.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_load.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_load.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_load.add(ZeroPadding2D((1, 1)))
model_load.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_load.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model_load.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model_load.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [64]:
## generate training and validation data (intermediate stage)

generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)
bottleneck_features_train = model_load.predict_generator(generator, nb_train_samples)
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model_load.predict_generator(generator, nb_validation_samples)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

Found 2400 images belonging to 24 classes.
Found 240 images belonging to 24 classes.


In [67]:
## load training and validation data
## create labels
train_data = np.load(open('bottleneck_features_train.npy'))
train_labels = np.array([])
for i in range(nb_category):
    current = np.array([i] * 100).astype(int)
    train_labels = np.append(train_labels, current)
train_labels = to_categorical(train_labels)


validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([])
for i in range(nb_category):
    current = np.array([i] * 10).astype(int)
    validation_labels = np.append(validation_labels, current)
validation_labels = to_categorical(validation_labels)

In [68]:
## train and save weights
model_train = Sequential()
model_train.add(Flatten(input_shape=train_data.shape[1:]))
model_train.add(Dense(256, activation='relu'))
model_train.add(Dropout(0.5))
model_train.add(Dense(nb_category, activation='softmax'))
model_train.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_train.fit(train_data, train_labels,
          nb_epoch=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))
model_train.save_weights(top_model_weights_path)

Train on 2400 samples, validate on 240 samples
Epoch 1/50
2400/2400 [==============================] - 5s - loss: 0.2254 - acc: 0.9543 - val_loss: 0.1623 - val_acc: 0.9597
Epoch 2/50
2400/2400 [==============================] - 5s - loss: 0.1496 - acc: 0.9601 - val_loss: 0.1376 - val_acc: 0.9609
Epoch 3/50
2400/2400 [==============================] - 5s - loss: 0.1344 - acc: 0.9624 - val_loss: 0.1242 - val_acc: 0.9639
Epoch 4/50
2400/2400 [==============================] - 6s - loss: 0.1226 - acc: 0.9638 - val_loss: 0.1263 - val_acc: 0.9627
Epoch 5/50
2400/2400 [==============================] - 11s - loss: 0.1136 - acc: 0.9651 - val_loss: 0.1145 - val_acc: 0.9656
Epoch 6/50
2400/2400 [==============================] - 14s - loss: 0.1057 - acc: 0.9668 - val_loss: 0.1189 - val_acc: 0.9642
Epoch 7/50
2400/2400 [==============================] - 14s - loss: 0.0986 - acc: 0.9682 - val_loss: 0.1145 - val_acc: 0.9655
Epoch 8/50
2400/2400 [==============================] - 15s - loss: 0.0923 

In [194]:
##0 save model_load and model_train
with open('model_load.pkl', 'w') as f:
    pickle.dump(model_load, f)

with open('model_train.pkl', 'w') as f:
    pickle.dump(model_train, f)

In [69]:
## define look-up table
lookup = {0:'beach', 1:'bridge', 2:'cavern', 3:'cliff', 4:'dam', 5:'desert', 6:'farmland', 7:'glacier', 8:'island', 9:'lake', 10:'market', 11:'marsh', 12:'monument', 13:'mountain', 14:'ocean', 15:'prairie', 16:'rainforest', 17:'road', 18:'skyscraper', 19:'stadium', 20:'taiga', 21:'tundra', 22:'volcano', 23:'waterfall'}

In [70]:
## load a test image and predict
im = cv2.resize(cv2.imread('./test_images/IMG_9984.JPG'), (img_width, img_height)).astype(np.float32)
im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)
im_transform = model_load.predict(im*1./255)
predict_proba = model_train.predict(im_transform)
lookup[np.argmax(predict_proba)]

'stadium'

In [14]:
## load a test image and predict
im = cv2.resize(cv2.imread('./test_images/IMG_6183.JPG'), (img_width, img_height)).astype(np.float32)
#im = cv2.resize(cv2.imread('./train_images_rename/lake/lake_060.jpg'), (img_width, img_height)).astype(np.float32)
im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)
im_transform = model_load.predict(im*1./255) # not so sure, may not contain all transformations in a image generator
predict_proba = model_train.predict(im_transform)
locs = map(lambda x:lookup[x], range(nb_category))
proba_order = zip(predict_proba.tolist()[0], locs)
sorted(proba_order, reverse=True)

[(0.935137152671814, 'desert'),
 (0.05370878055691719, 'ocean'),
 (0.008119584992527962, 'prairie'),
 (0.0013521823566406965, 'island'),
 (0.0006508431979455054, 'beach'),
 (0.00034779563429765403, 'cliff'),
 (0.00017916649812832475, 'tundra'),
 (0.00010392873809905723, 'dam'),
 (7.946159166749567e-05, 'bridge'),
 (6.385222513927147e-05, 'volcano'),
 (5.721237539546564e-05, 'skyscraper'),
 (4.743493263958953e-05, 'mountain'),
 (3.807663597399369e-05, 'glacier'),
 (3.705767812789418e-05, 'farmland'),
 (2.5708570319693536e-05, 'marsh'),
 (1.4255322639655787e-05, 'road'),
 (1.2928856449434534e-05, 'rainforest'),
 (6.956678589631338e-06, 'monument'),
 (6.72828127790126e-06, 'cavern'),
 (4.862476544076344e-06, 'lake'),
 (2.8137317258369876e-06, 'stadium'),
 (2.2937781523069134e-06, 'taiga'),
 (8.669925932736078e-07, 'waterfall'),
 (5.490556631571053e-08, 'market')]

In [182]:
##2 generate testing data(user image--intermediate stage)
test_data_dir = './test_images/'
test_data_gen_folder = './test_images/test/' # there must be a subfolder(generator requirement)
test_data_gen = os.listdir(test_data_gen_folder)
test_data_gen = [test_d for test_d in test_data_gen if ('.jpg' in test_d)|('.JPG' in test_d)]
nb_test_images = len(test_data_gen)

generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

test_transformed = model_load.predict_generator(generator, nb_test_images)

##2 predict using image-generator instead of direct loading
test_transformed_sg = test_transformed[1,:,:,:]
test_transformed_sg = np.expand_dims(test_transformed_sg, axis=0)
predict_proba = model_train.predict(test_transformed_sg)
locs = map(lambda x:lookup[x], range(nb_category))
proba_order = zip(predict_proba.tolist()[0], locs)
sorted(proba_order, reverse=True)

Found 2 images belonging to 1 classes.


In [180]:
##3 multiple images generate testing data(user image--intermediate stage)
test_data_dir = './test_images/'
test_data_gen_folder = './test_images/test/' # there must be a subfolder(generator requirement)
test_data_gen = os.listdir(test_data_gen_folder)
test_data_gen = [test_d for test_d in test_data_gen if ('.jpg' in test_d)|('.JPG' in test_d)]
nb_test_images = len(test_data_gen)

generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

test_transformed = model_load.predict_generator(generator, nb_test_images)

Found 2 images belonging to 1 classes.


In [181]:
##3 multiple images average predict using image-generator instead of direct loading

mp_predict_proba = np.zeros([nb_test_images, nb_category])

for i in range(nb_test_images):
    test_transformed_sg = test_transformed[i,:,:,:]
    test_transformed_sg = np.expand_dims(test_transformed_sg, axis=0)
    predict_proba = model_train.predict(test_transformed_sg)
    mp_predict_proba[i,:] = predict_proba
    
avg_predict_proba = mp_predict_proba.mean(axis = 0)
locs = map(lambda x:lookup[x], range(nb_category))
proba_order = zip(avg_predict_proba.tolist(), locs)
sorted(proba_order, reverse=True)

[(0.4998003264654751, 'rainforest'),
 (0.35477447509765625, 'tundra'),
 (0.14030525088313478, 'desert'),
 (0.004603391978907327, 'glacier'),
 (0.00024399514951888902, 'ocean'),
 (0.00019465064691383867, 'waterfall'),
 (4.465589154281345e-05, 'prairie'),
 (1.2171853995826107e-05, 'cliff'),
 (7.1141480794640485e-06, 'mountain'),
 (6.052084057461826e-06, 'taiga'),
 (4.8336711921947995e-06, 'cavern'),
 (1.2718490214592114e-06, 'beach'),
 (9.364486004488272e-07, 'farmland'),
 (5.125231423610665e-07, 'lake'),
 (1.9560242846619613e-07, 'market'),
 (9.730300295184013e-08, 'road'),
 (4.2022364706018056e-08, 'volcano'),
 (1.6827681498026922e-08, 'bridge'),
 (1.5076273005959598e-08, 'dam'),
 (7.390213330182865e-09, 'marsh'),
 (2.289043787200365e-09, 'island'),
 (2.0338654857510186e-09, 'skyscraper'),
 (7.537537822366328e-10, 'stadium'),
 (5.340672452891493e-10, 'monument')]